# Travaille Pratique 
Nom: Gustavo BARRETO

## Exercice 1 - MandelBrot Set


In [19]:
import subprocess
import pandas as pd

### Function Declarations

In [3]:
def get_avg_times(logfile: str) -> pd.DataFrame:
    df = pd.read_csv(logfile, sep=";", engine="python", header=None, names=["PROC","OP", "TIME_MS"])
    df['TIME_MS'] = df['TIME_MS'].astype(float)
    
    return df

def run_mpi(repeats=5, np=4, script="mandelbrot-mpi.py", logfile="mandelbrot-mpi.log"):
    for _ in range(repeats):
        subprocess.run(
            ["mpiexec", "-np", str(np), "python", script],
            stdout=open(logfile, "a"),
            stderr=subprocess.STDOUT
        )

def run_serial(repeats=5, script="mandelbrot-serial.py", logfile="mandelbrot-serial.log"):
    for _ in range(repeats):
        subprocess.run(
            ["python", script],
            stdout=open(logfile, "a"),
            stderr=subprocess.STDOUT
        )
        
    


To execute the code:

## Question 1 - implementation statique en colonne

In [10]:
run_mpi(repeats=10, np=8, script="mandelbrot-mpi.py", logfile="timings.log")
# run_serial(repeats=5, script="mandelbrot.py", logfile="timings.log")

In [ ]:
#Obtain the results from the executions
df = get_avg_times("timings.log")

# Display the average times per operation
avg = df.groupby(["OP","PROC"], as_index=False)[["TIME_MS"]].mean()
display(avg)


# Calculate speedup
serial_time = avg.loc[avg["OP"] == "Total(Serial)", "TIME_MS"].values[0]
mpi_time = avg.loc[avg["OP"] == "Total", "TIME_MS"].min()
speedup = serial_time / mpi_time
print(f"Speedup (Serial / MPI): {speedup:.2f}x")



,OP,PROC,TIME_MS
0,Calcul,2,1151.167950
1,Calcul,4,638.416614
2,Calcul,8,677.159037
3,Calcul,16,657.604213
4,Total,2,1187.481200
5,Total,4,705.594485
6,Total,8,871.733300
7,Total,16,1255.424600
8,Total(Serial),1,2320.822400


Speedup (Serial / MPI): 3.29x


La parallélisation réduit fortement le temps d’exécution par rapport au cas séquentiel, avec un speed-up maximal de 3,29 obtenu pour 4 processus. En revanche, l’augmentation du nombre de processus au-delà de cette valeur n’apporte plus de gain et peut même dégrader les performances, en raison du surcoût des communications et des synchronisations MPI. Ces résultats illustrent la loi d’Amdahl et montrent que le speed-up est limité pour un problème de taille modérée.

## Utilisation d'une nouvelle strategie: Slave-Master

In [17]:
for i in [4, 8, 16]:
    run_mpi(repeats=5, np=i, script="mandelbrot-SlaveMaster.py", logfile="slave-master.log")
run_serial(repeats=3, script="mandelbrot.py", logfile="slave-master.log")


In [ ]:
df = get_avg_times("slave-master.log")

# Display the average times per operation
avg = df.groupby(["OP","PROC"], as_index=False)[["TIME_MS"]].mean()
display(avg)



# Calculate speedup
serial_time = avg.loc[avg["OP"] == "Total(Serial)", "TIME_MS"].values[0]
mpi_time = avg.loc[avg["OP"] == "Total", "TIME_MS"].min()
speedup = serial_time / mpi_time
print(f"Speedup (Serial / MPI): {speedup:.2f}x")




,OP,PROC,TIME_MS
0,Total,4,2987.3826
1,Total,8,2149.8630
2,Total,16,1415.4264
3,Total(Serial),1,8177.4710


Speedup (Serial / MPI): 5.78x


Cette stratégie maître–esclave est plus efficace car elle répartit dynamiquement les tâches selon la disponibilité des workers. Chaque processus reçoit un bloc de lignes à calculer et, dès qu’il termine, il obtient un nouveau bloc. Ainsi, aucun worker n’est inactif, même si certaines lignes sont plus complexes à calculer. Cette approche assure un load balancing optimal, réduit le temps total de calcul et évite que la performance soit limitée par les tâches les plus coûteuses, contrairement à une distribution statique.

# Produit Matrice-Vecteur


In [23]:
for i in [4, 8, 16]:
    run_mpi(repeats=6, np=i, script="matvec-col.py", logfile="matvec-col.log")
    run_mpi(repeats=6, np=i, script="matvec-row.py", logfile="matvec-row.log")

run_serial(repeats=3, script="matvec.py", logfile="matvec-col.log")
run_serial(repeats=3, script="matvec.py", logfile="matvec-row.log")

Analise du paralelisme en colonne:

In [25]:
df = get_avg_times("matvec-col.log")
avg = df.groupby(["OP","PROC"], as_index=False)[["TIME_MS"]].mean()
display(avg)


# Calculate speedup
serial_time = avg.loc[avg["OP"] == "Total(Serial)", "TIME_MS"].values[0]
mpi_time = avg.loc[avg["OP"] == "Total", "TIME_MS"].min()
speedup = serial_time / mpi_time
print(f"Speedup (Serial / MPI): {speedup:.2f}x")


,OP,PROC,TIME_MS
0,Total,4,0.843222
1,Total,8,7.655444
2,Total,16,39.521889
3,Total(Serial),1,27.828167


Speedup (Serial / MPI): 33.00x


Analise du paralelisme en ligne:

In [28]:
df = get_avg_times("matvec-row.log")
avg = df.groupby(["OP","PROC"], as_index=False)[["TIME_MS"]].mean()
display(avg)


# Calculate speedup
serial_time = avg.loc[avg["OP"] == "Total(Serial)", "TIME_MS"].values[0]
mpi_time = avg.loc[avg["OP"] == "Total", "TIME_MS"].min()
speedup = serial_time / mpi_time
print(f"Speedup (Serial / MPI): {speedup:.2f}x")


,OP,PROC,TIME_MS
0,Total,4,8.606222
1,Total,8,15.062667
2,Total,16,75.062000
3,Total(Serial),1,28.182667


Speedup (Serial / MPI): 3.27x


Les résultats montrent que la parallélisation réduit significativement le temps d’exécution par rapport au séquentiel. Pour la stratégie par lignes, le temps total diminue jusqu’à 8 processus, mais augmente pour 16 à cause du surcoût de communication. Pour la stratégie par colonnes, le gain est encore plus limité avec l’augmentation du nombre de processus, car chaque processus doit calculer des contributions partielles pour toutes les lignes, entraînant davantage de communications. Les deux approches accélèrent le calcul, mais l’efficacité dépend du nombre de processus et de la répartition du travail.

<!-- $$

S_{max} = \frac{1}{1-f + f/n} \rightarrow^{n >> 1} \frac{1}{1-0.9} = 10
$$ -->